In [15]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
 
import torch
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size=64
learning_rate = 0.1
layers = 100
 
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
 
transform_test = transforms.Compose([
    transforms.ToTensor()])
 
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data',train = True, download = True, transform = transform_train), batch_size = batch_size, shuffle = True
)
 
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('../data', train = False, transform = transform_test), batch_size = batch_size,shuffle = True
)

         
class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BottleneckBlock,self).__init__()
        inter_planes = out_planes * 4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace = True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size = 1, stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(inter_planes)
        self.conv2 = nn.Conv2d(inter_planes, out_planes, kernel_size = 3, stride = 1, padding = 0, bias = False)
        self.droprate = dropRate
         
    def forward(self,x):
        out = self.conv1(self.relu(self.bn1(x)))
        out = self.conv2(self.relu(self.bn2(out)))
        
        return torch.cat([x,out],1)


class DenseBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, growh_rate, block):
        super(DenseBlock,self).__init__()
        self.layer = self._make_layer(block, in_planes, growh_rate, nb_layers)
     
    def _make_layer(self,block,in_planes,growh_rate,nb_layers):
        layers=[]
        for i in range(nb_layers):
            layers.append(block(in_planes + i * growh_rate ,growh_rate))
             
        return nn.Sequential(*layers)
     
    def forward(self,x):
        return self.layer(x)


class TransitionBlock(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(TransitionBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace = True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size = (1, 1), stride = 1, padding = 0, bias = False)
         
    def forward(self,x):
        out = self.conv1(self.relu(self.bn1(x)))

        return F.avg_pool2d(out,2)


class DenseNet(nn.Module):
    def __init__(self, depth, num_classes, growh_rate = 12):
        super(DenseNet,self).__init__()
        num_of_blocks = 4
        in_planes = 16 # 2 * growh_rate
        n = (depth - num_of_blocks - 1)/num_of_blocks
        in_planes = 2 * growh_rate
        block = BottleneckBlock
         
        n = int(n)
#         self.conv1 = nn.Conv2d(3,in_planes,kernel_size=3,stride=1,padding=1,bias=False) 
        self.conv1 = nn.Conv2d(3, in_planes, kernel_size = (7, 7), stride = 2, padding = 3, bias = False)
        self.block1 = DenseBlock(n, in_planes, growh_rate, block)
        self.trans1 = TransitionBlock(in_planes, in_planes)
        self.block2 = DenseBlock(n, in_planes, growh_rate, block)
        self.trans2 = TransitionBlock(in_planes, in_planes)
        self.block3 = DenseBlock(n, in_planes, growh_rate, block)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace = True)
        self.fc = nn.Linear(in_planes, num_classes)
        self.in_planes = in_planes
         
    def forward(self,x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.trans1(out)
        out = self.block2(out)
        out = self.trans2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out,8)
        out = out.view(-1, self.in_planes) 
        return self.fc(out)

model = DenseNet(layers,10, growh_rate = 12)
model = model.to(device)
 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9)


def train(train_loader,model,criterion,optimizer,epoch):
    model.train()
    for i, (input,target) in enumerate(train_loader):
        target = target.to(device)
        input = input.to(device)
         
        output = model(input)
        loss = criterion(output,target)
         
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
         
        if(i%20 == 0):
            print("loss in epoch %d , step %d : %f" % (epoch, i,loss.data[0]))


def test(test_loader,model,criterion,epoch):
    model.eval()
    correct = 0
     
    for i, (input,target) in enumerate(test_loader):
        target = target.to(device)
        input = input.to(device)
         
        output = model(input)
        loss = criterion(output,target)
         
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().float().sum()
     
    print("Accuracy in epoch %d : %f" % (epoch,100.0*correct/len(test_loader.dataset)))

def adjust_lr(optimizer, epoch, learning_rate):
    if epoch==150 :
        learning_rate*=0.1
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    
for epoch in range(0,300):
    adjust_lr(optimizer,epoch,learning_rate)
    train(train_loader,model,criterion,optimizer,epoch)
    test(test_loader,model,criterion,epoch)

Files already downloaded and verified


RuntimeError: running_mean should contain 300 elements not 24